In [ ]:
//.server.js

const http = require("node:http");
const jsonDB = require("./utils/jsonDB");

// Tạo biến db
let db = {};

// Đọc db từ utils
jsonDB.read().then((result) => {
    db = result;
});

// Mặc dù là bất đồng bộ nhưng việc đọc file lúc nào cũng nhanh hơn request API lên

/**
 * Xử lý CORS, Gọi API
 */

const allowOrigins = ["http://localhost:5173", "http://localhost:5174"];

const server = http.createServer((req, res) => {
    /* Xử lý CORS */
    const rqOrigin = req.headers.origin;
    const validOrigin = allowOrigins.find((_origin) => _origin === rqOrigin);

    const headers = {
        "Content-Type": "application/json",
    };

    if (validOrigin) headers["Access-Control-Allow-Origin"] = validOrigin;

    /* [GET] /api/tasks */
    if (req.method === "GET" && req.url === "/api/tasks") {
        res.writeHead(200, headers);
        res.end(JSON.stringify(db.tasks));
        return;
    }

    /* [POST] /api/tasks */
    if (req.method === "POST" && req.url === "/api/tasks") {
        let body = "";
        req.on("data", (chunk) => {
            body += chunk.toString();
            console.log("Chunk: ", chunk.toString());
        });
        req.on("end", () => {
            const payload = JSON.parse(body);
            const maxId = Math.max(...db.tasks.map((t) => t.id));
            const newTask = {
                id: maxId + 1,
                title: payload.title,
            };

            db.tasks.push(newTask);
            // Lưu db / ghi đè db
            jsonDB.save(db);
            res.writeHead(201, headers);
            res.end(
                JSON.stringify({
                    data: newTask,
                })
            );
        });

        return;
    }

    /* [PUT/PATCH] /api/tasks */
    if (
        ["PUT", "PATCH"].includes(req.method) &&
        req.url.startsWith("/api/tasks/")
    ) {
        res.writeHead(200, headers);
        res.end(
            JSON.stringify({
                status: "SUCCESS",
            })
        );
        return;
    }

    /* CORS: Preflight Request */
    if (req.method === "OPTIONS") {
        headers["Access-Control-Allow-Methods"] = "PUT,PATCH,DELETE";
        headers["access-control-max-age"] = 10; //Cache
        res.writeHead(200, headers);
        res.end("OPTIONS");
        return;
    }

    /* Bypass-CORS */
    if (req.url.startsWith("/bypass-cors")) {
        // Gọi API từ Client: http://localhost:3000/bypass-cors?url=https://api-gateway.f8.edu.vn/api/assignment-classes/stats

        // 1. Lấy API Url từ phía Client
        const queryString = req.url.split("?").pop();
        const param = new URLSearchParams(queryString);
        const originUrl = param.get("url");

        // 2. Gọi API tại NodeJS không bị CORS
        fetch(originUrl, {
            method: req.method,
        })
            .then((response) => {
                headers["Content-Type"] = response.headers.get("Content-Type");
                return response.text();
            })
            .then((result) => {
                res.writeHead(200, headers);
                res.end(result);
            });
        return;
    }

    /* 404 Not Found */
    res.writeHead(404, { "Content-Type": "text/plain" });
    res.end("Not Found");
    return;
});

server.listen(3000, () => {
    console.log("Máy chủ đang chạy...");
});


In [ ]:
//utils/jsonDB.js

const fs = require("node:fs/promises");
const filePath = "./db.json";

/* Đọc file */
async function read() {
    try {
        const result = await fs.readFile(filePath, "utf-8");
        return JSON.parse(result);
    } catch (error) {
        if (error.code === "ENOENT") {
            // todo: Ghi file: {}
            // Nếu file db.json thì chúng ta tạo file đó.
            // Nội dung trong file db.json mặc định là {} rỗng.
            const defaultDB = {};
            save(defaultDB); // Mặc định
            return defaultDB;
        }
    }
}

/* Ghi file */
async function save(db) {
    try {
        await fs.writeFile(filePath, JSON.stringify(db, null, 4), "utf-8");
    } catch (error) {
        // Lỗi liên quan đến hệ thống
        console.log(error);
    }
}

/* Export */
module.exports = { read, save };


In [ ]:

{
    "tasks": [
        {
            "id": 1,
            "title": "Nấu cơm"
        },
        {
            "id": 2,
            "title": "Quét nhà"
        },
        {
            "id": 3,
            "title": "Rua bat"
        },
        {
            "id": 4,
            "title": "Rua bat"
        }
    ]
}